In [40]:
import pandas as pd
import numpy as np
import math
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [41]:
df = pd.read_csv("/Users/florian/Documents/GitHub/4 Semester/APPD/Analyse_von_Prozessdaten/setup/SmA-Four-Tank-Batch-Process_V2.csv", sep= ";")

In [42]:
df = df.drop(["timestamp"], axis= 1)

In [59]:
df = df.astype(float)

In [54]:
X = df.drop(["DeviationID ValueY", "CuStepNo ValueY"], axis=1)
y= df["DeviationID ValueY"]

In [60]:
df

,DeviationID ValueY,CuStepNo ValueY,LevelMainTank ValueY,YC10001_MV ValueY,PI12002_PV_Out ValueY,PI12003_PV_Out ValueY,YC14001_MV ValueY,FIC14002_MV ValueY,FIC14002_PV_Out ValueY,FIC14002_SP ValueY,...,LIC22002_PV_Out ValueY,LIC22002_SP ValueY,LIC21002_MV ValueY,LIC21002_PV_Out ValueY,LIC21002_SP ValueY,PIC14007_MV ValueY,PIC14007_PV_Out ValueY,PIC14007_SP ValueY,YC14008_Ctrl ValueY,PI10003_PV_Out ValueY
0,1.0,1.0,149.985779,100.0,0.070602,0.087963,100.0,100.0,0.0,2000.0,...,1.966218,2.0,0.611938,1.979745,15.0,10.0,0.127008,0.2,0.0,0.001710
1,1.0,1.0,149.985779,100.0,0.070602,0.086806,100.0,100.0,0.0,2000.0,...,1.967810,2.0,1.151489,1.979745,15.0,10.0,0.127021,0.2,0.0,0.001715
2,1.0,1.0,150.027740,100.0,0.071759,0.103588,100.0,100.0,0.0,2000.0,...,1.958261,2.0,11.073669,1.986111,15.0,10.0,0.127021,0.2,0.0,0.001712
3,1.0,1.0,150.043427,100.0,0.072338,0.173611,100.0,100.0,0.0,2000.0,...,1.958261,2.0,11.819598,1.982133,15.0,10.0,0.126945,0.2,0.0,0.001698
4,1.0,1.0,150.015564,100.0,0.065972,0.326968,100.0,100.0,0.0,2000.0,...,1.967810,2.0,22.069590,1.982133,15.0,10.0,0.126945,0.2,0.0,0.001700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278487,9.0,8.0,112.293930,100.0,-0.899306,-0.582176,100.0,100.0,0.0,2000.0,...,6.972077,2.0,800.000000,7.470197,15.0,10.0,-0.009348,0.2,0.0,-0.009062
278488,9.0,8.0,112.352432,100.0,-0.902199,-0.586227,100.0,100.0,0.0,2000.0,...,6.968894,2.0,800.000000,7.478154,15.0,10.0,-0.009345,0.2,0.0,-0.009085
278489,9.0,8.0,112.400436,100.0,-0.901042,-0.565394,100.0,100.0,0.0,2000.0,...,6.961733,2.0,800.000000,7.471788,15.0,10.0,-0.009364,0.2,0.0,-0.009090
278490,9.0,8.0,112.441315,100.0,-0.902778,-0.578704,100.0,100.0,0.0,2000.0,...,6.957755,2.0,800.000000,7.475767,15.0,10.0,-0.009384,0.2,0.0,-0.009099


In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.3)

In [80]:
def lvq_train(input_data, labels, a, b, max_epochs, min_a, e):
    classes, train_idx = np.unique(labels, return_index=True)
    prototypes = input_data.iloc[train_idx].astype(np.float64)
    train_data = np.array([e for i, e in enumerate(zip(input_data, labels)) if i not in train_idx])
    input_data = train_data[:, 0]
    labels = train_data[:, 1]
    epochs = 0
 
    while epochs < max_epochs and a > min_a:
        for i, x in enumerate(input_data):
            distances = [np.sqrt(np.sum((w - x) ** 2)) for w in prototypes]
            min_1 = np.argmin(distances)
            min_2 = distances.index(sorted(distances)[1])
            dc = float(np.amin(distances))
            dr = float(distances[min_2])
 
            if labels[i] == classes[min_1]:
                prototypes[min_1] = prototypes[min_1] + a * (x - prototypes[min_1])
            elif labels[i] == classes[min_2] and classes[min_1] != classes[min_2]:
                if dc <= (1+e) * dr and dr <= (1+e) * dc:
                    prototypes[min_1] = prototypes[min_1] - a * (x - prototypes[min_1])
                    prototypes[min_2] = prototypes[min_2] + a * (x - prototypes[min_2])
            elif labels[i] == classes[min_2] and classes[min_1] == classes[min_2]:
                prototypes[min_1] = prototypes[min_1] + e * a * (x - prototypes[min_1])
                prototypes[min_2] = prototypes[min_2] + e * a * (x - prototypes[min_2])
 
        a *= b
        epochs += 1
 
    return prototypes, classes

def lvq_test(x, W):
    
    W, c = W
    d = [math.sqrt(sum((w - x) ** 2)) for w in W]
 
    return c[np.argmin(d)]

In [81]:
W = lvq_train(X_train, y_train, 0.2, 0.5, 100, 0.001, 0.3)
W

TypeError: unsupported operand type(s) for -: 'str' and 'numpy.str_'

In [ ]:
predicted = []
for i in X_test:
    predicted.append(lvq_test(i, W))

In [ ]:
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels,
           preds, average = 'weighted')))
    print("Recall Score: {}".format(recall_score(labels, preds,
           average = 'weighted')))
    print("Accuracy Score: {}".format(accuracy_score(labels,
           preds)))
    print("F1 Score: {}".format(f1_score(labels, preds, average =
           'weighted')))
print_metrics(y_test, predicted)